# FastText

FastText是由Facebook的人工智能研究团队开发的一种用于文本分类和词向量表示的开源库，适用于大规模数据集和高性能需求的任务。FastText的相关论文由Piotr Bojanowski等人在2016年发表，FastText的作者认为之前的文本向量化方法没有考虑单词内部的子词之间的关系，所以性能受限。为了改善这个限制，FastText的核心原理包括两大部分：**词向量表示和文本分类**。

## 词向量表示

FastText 的一个关键创新在于利用了子词信息（Subword Information），即将每个单词表示为多个 n-gram 的组合。


例如，单词 "where" 可以分解为"wh", "whe", "her", "ere", "re"等 n-gram。这种方法使得模型能够捕捉到词的子词信息，从而更好地处理未登录词和拼写错误。FastText 使用基于在上文中提到的Skip-Gram的方法来训练embedding。值得注意的是，FastText不仅使用单词本身，还使用单词的所有n-gram。每个单词被表示为它所有n-gram向量的组合。为了提高计算效率，FastText 使用分层Softmax代替传统的Softmax。分层Softmax将所有类别组织成一棵二叉树，每个叶节点对应一个类别。通过减少类别数量的对数级计算，分层Softmax极大地加速了训练和预测过程。

## 文本分类原理

在文本分类任务中，FastText通过将文本表示为一系列词向量的平均来生成文档向量。对于一个文本d中的每个词w，其向量表示为vw。文本d的表示为所有词向量的平均，即：
$$
v_d = \frac{1}{|d|} \sum_{w \in d} v_w
$$
> 使用fastText进行文本分类的同时也会产生词的embedding，即embedding是fastText分类的产物

FastText使用线性分类器来进行文本分类。文本向量vd通过一个全连接层，并使用Softmax函数来输出类别概率。分类模型的公式为
$$
y=Softmax(W \cdot v_d + b)
$$
其中W是权重矩阵，b是偏置项

### **fastText文本分类的核心思想是什么？**

仔细观察模型的后半部分，即从隐含层输出到输出层输出，会发现它就是一个softmax线性多类别分类器，分类器的输入是一个用来表征当前文档的向量；模型的前半部分，即从输入层输入到隐含层输出部分，主要在做一件事情：生成用来表征文档的向量。那么它是如何做的呢？叠加构成这篇文档的所有词及n-gram的词向量，然后取平均。叠加词向量背后的思想就是传统的词袋法，即将文档看成一个由词构成的集合。
于是fastText的核心思想就是：**将整篇文档的词及n-gram向量叠加平均得到文档向量，然后使用文档向量做softmax多分类。这中间涉及到两个技巧：字符级n-gram特征的引入以及分层Softmax分类。**

### FastText与Word2Vec的区别？

● **词汇表外（OOV）单词的处理**：
Word2Vec： Word2Vec 在单词级别运行，为各个单词生成嵌入。它难以处理词汇表之外的单词，因为它无法代表训练期间未见过的单词。\
fastText：相比之下，fastText 通过将单词视为由字符 n-gram 组成来引入子词嵌入。这使得它能够通过将术语分解为子词单元并为这些单元生成嵌入来有效地处理词汇表外的单词，甚至对于没有见过的单词也是如此。此功能使 fastText 在处理罕见或形态复杂的表达式时更加稳健。

● **单词的表示**
Word2Vec： Word2Vec 仅基于单词生成单词嵌入，而不考虑内部结构或形态信息 \
fastText： fastText 捕获子词信息，使其能够根据组成字符 n 元语法来理解单词含义。这使得 fastText 能够通过考虑单词的形态构成来表示单词，从而提供更丰富的表示，特别是对于形态丰富的语言或具有专门术语的领域。

● **训练效率**
Word2Vec： Word2Vec 中的训练过程比旧方法相对更快，但由于其字级方法，可能比 fastText 慢。 \
fastText： fastText 以其卓越的速度和可扩展性而闻名，特别是在处理大型数据集时，因为它在子词级别上高效运行。

● **使用场景**
Word2Vec： Word2Vec 的单词级嵌入非常适合查找相似单词、理解单词之间的关系以及捕获语义相似性等任务 \
fastText： fastText 的子词嵌入使其更适合涉及词汇外单词、情感分析、语言识别以及需要更深入理解形态的任务的场景。
## 优缺点
优点：

● 训练和预测速度也很快，适合实时应用。
● 能够处理具有大量类别的任务，并保持高效性。
● 支持多种语言的文本处理。

缺点：
● 使用静态词向量，无法捕捉上下文相关的词义变化。

## 应用场景
● 文本分类
● 关键词提取
● 拼写错误纠正

In [2]:
from gensim.models.fasttext import FastText
import jieba
import re
import os
import time

In [3]:
def read_stop(stop_words_path):
    """
    读取停用词
    :return:
    """
    # 读取停用词
    stop_words = []
    with open(stop_words_path, "r", encoding="utf-8") as f_reader:
        for line in f_reader:
            line = line.replace("\r", "").replace("\n", "").strip()
            stop_words.append(line)
    # print(len(stop_words))
    stop_words = set(stop_words)
    # print(len(stop_words))
    return stop_words

In [4]:
def data_process(stop_words, data_path, split_data_path):
    """
    数据预处理
    :return:
    """

    # 文本预处理
    sentecnces = []
    rules = u"[\u4e00-\u9fa5]+"
    pattern = re.compile(rules)
    f_writer = open(split_data_path, "w", encoding="utf-8")

    with open(data_path, "r", encoding="utf-8") as f_reader:
        for line in f_reader:
            line = line.replace("\r", "").replace("\n", "").strip()
            if line == "" or line is None:
                continue
            line = " ".join(jieba.cut(line))
            seg_list = pattern.findall(line)
            word_list = []
            for word in seg_list:
                if word not in stop_words:
                    word_list.append(word)
            if len(word_list) > 0:
                sentecnces.append(word_list)
                line = " ".join(word_list)
                f_writer.write(line + "\n")
                f_writer.flush()
    f_writer.close()
    return sentecnces

In [7]:
dir_current = os.getcwd()
# 注意此处的路径，在 jupyter_notebook_config.py 中设置，否则会报错
stop_words_file = os.path.join(dir_current, "learning_word-vector/data/stop_words.txt")
data_file = os.path.join(dir_current, "learning_word-vector/data/天龙八部.txt")
split_data_file = os.path.join(dir_current, "learning_word-vector/data/天龙八部_split.txt")

stop_words = read_stop(stop_words_file)
sentences = data_process(stop_words, data_file, split_data_file)
print(len(sentences))
sentences[:10]

10961


[['书名', '天龙八部'],
 ['作者', '金庸'],
 ['本文', '早安', '电子书', '网友', '分享', '版权', '原作者'],
 ['用于', '商业行为', '后果自负'],
 ['早安', '电子书'],
 ['金庸', '作品集', '三联', '版', '序'],
 ['小学',
  '时',
  '爱读',
  '课外书',
  '低年级',
  '时看',
  '儿童',
  '画报',
  '小朋友',
  '小学生',
  '内容',
  '小朋友',
  '文库',
  '似懂非懂',
  '阅读',
  '各种各样',
  '章回小说',
  '五六年',
  '级',
  '时',
  '看新',
  '文艺作品',
  '喜爱',
  '古典文学',
  '作品',
  '多于',
  '近代',
  '当代',
  '新文学',
  '个性',
  '使然',
  '朋友',
  '喜欢',
  '新文学',
  '不爱',
  '古典文学'],
 ['知识',
  '当代',
  '书报',
  '中',
  '寻求',
  '小学',
  '时代',
  '得益',
  '记忆',
  '最深',
  '爸爸',
  '哥哥',
  '购置',
  '邹韬奋',
  '所撰',
  '萍踪',
  '寄语',
  '萍踪',
  '忆语',
  '世界各地',
  '旅行',
  '记',
  '主编',
  '生活',
  '周报',
  '新',
  '旧',
  '童年时代',
  '深受',
  '邹先生',
  '生活',
  '书店',
  '之惠',
  '生活',
  '书店',
  '三联书店',
  '组成部分',
  '十多年',
  '前',
  '香港三联书店',
  '签',
  '合同',
  '中国',
  '大陆',
  '地区',
  '出版',
  '小说',
  '因事',
  '未果',
  '重',
  '行',
  '筹划',
  '三联书店',
  '独家',
  '出版',
  '中国',
  '大陆',
  '地区',
  '简体字',
  '感到',
  '欣慰',
  '回忆',
  '昔日',
  '心中',
  '充满',
  '温馨',
  '之

In [8]:
# 模型训练
start_time = time.time()
print("开始训练模型...")
SG = 1 # 0表示CBOW，1表示Skip-gram

model = FastText(sentences, vector_size=128, epochs=50, window=5, min_count=6,
                          workers=10, sg=SG,
                          negative=5, hs=0, seed=42,
                          )

print("训练模型结束，耗时：", time.time() - start_time)
# 保存模型
if SG == 0:
    model_name = "天龙八部_fasttext_CBOW.bin"
else:
    model_name = "天龙八部_fasttext_Skip-gram.bin"

model_path = os.path.join(dir_current, f"learning_word-vector/models/{model_name}")
print("保存模型到：", model_path)

model.save(model_path)

开始训练模型...
训练模型结束，耗时： 24.37697434425354
保存模型到： E:\Pycharm_Data\Pycharm_Project\MyNLP_Project\NLP-Learning-Workshop\learning_word-vector/models/天龙八部_fasttext_Skip-gram.bin


In [10]:
# 加载模型
model2 = FastText.load(model_path)

# 选出10个与乔峰最相近的10个词
for e in model2.wv.most_similar(positive=["乔峰"], topn=10):
    print(e[0], e[1])

# 计算两个词语的相似度
sim_value = model2.wv.similarity('乔峰', '萧峰')
print("乔峰与萧峰的相似度：", sim_value)

# 计算两个集合的相似度
list1 = ['乔峰', '萧远山']
list2 = ['慕容复', '慕容博']
sim_value = model2.wv.n_similarity(list1, list2)
print("['乔峰', '萧远山']与['慕容复', '慕容博']的相似度：\n\t", sim_value)

那厮 0.5562141537666321
徐长老 0.5247012972831726
乔大爷 0.5245589017868042
谭公 0.5162566304206848
鲍千灵 0.49004656076431274
鲍兄 0.48291927576065063
玄苦大师 0.46966931223869324
谭婆 0.4516589641571045
赵钱孙 0.44818931818008423
陈长老 0.44002845883369446
乔峰与萧峰的相似度： 0.36966228
['乔峰', '萧远山']与['慕容复', '慕容博']的相似度：
	 0.59094656


## 学习资源
● [FastText](https://fasttext.cc/)

● [FastText中文文档](https://fasttext.apachecn.org/#/)

● [fastText原理及实践](https://mp.weixin.qq.com/s/cnu2xDFgsXIT0ObaXfZYHw)

● [Welcome fastText to the Hugging Face Hub](https://huggingface.co/blog/fasttext)

## 代码学习

● [FastText](https://github.com/facebookresearch/fastText)
● [Word representations](https://fasttext.cc/docs/en/unsupervised-tutorial.html)

● [Word Embedding--Wiki(Word2vec、fastText)](https://github.com/lintseju/word_embedding)
